<a href="https://colab.research.google.com/github/Nawel-Bellil/AI---Deep-Learning--/blob/main/embedding_clusters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 📦 Import Libraries


In [1]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer
from sklearn.manifold import TSNE
import seaborn as sns
import requests
from io import StringIO
from sklearn.metrics import silhouette_score
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer


# 2. 🌐 Load Dataset




In [2]:
url = "https://raw.githubusercontent.com/susanli2016/PyCon-Canada-2019-NLP-Tutorial/master/bbc-text.csv"
response = requests.get(url)
data = pd.read_csv(StringIO(response.text))

In [3]:
print(f"Dataset shape: {data.shape}")
print(data['category'].value_counts())

Dataset shape: (2225, 2)
category
sport            511
business         510
politics         417
tech             401
entertainment    386
Name: count, dtype: int64


# 3. 🧪 Sample 100 Articles
### To keep the processing light and balanced

In [4]:
sample_size = 100
data_sample = data.groupby('category').apply(lambda x: x.sample(min(len(x), sample_size//5))).reset_index(drop=True)


<ipython-input-4-efe0a684d1d7>:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data_sample = data.groupby('category').apply(lambda x: x.sample(min(len(x), sample_size//5))).reset_index(drop=True)


# 4. ✂️ Preprocess Texts
### Keep only the first 20 words of each article (just enough to capture context, not too long).

### Convert into a list of short summaries.


In [5]:
texts = data_sample['text'].str.split().str[:20].str.join(' ').tolist()  # Get first 20 words
categories = data_sample['category'].tolist()

# 5. 🤖 Generate Sentence Embeddings ( semantic meaning )

In [6]:
print("Creating embeddings...")
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')  # A good model for semantic similarity
embeddings = model.encode(texts, show_progress_bar=True)

Creating embeddings...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.51k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]